In [ ]:
# IMPORT PACKAGES
import os
from numpy import zeros, array, average, genfromtxt, flip, savetxt, sort, unique, where, save, shape, linspace, rot90, fliplr, flipud
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime
import torch as nn

In [ ]:
# MAPPING FUNCTIONS
def nodalMapping(dataArray, numDim, numElem):
    ''' 
    Data import from FEM result
    '''
    numData     = len(dataArray[:,0])
    targetID    = dataArray[:,0]
    coordArray  = dataArray[:, 1:3]
    strainArray = dataArray[:, 5:9]
    if numDim == 2:
        dispArray   = dataArray[:,3:5]
    elif numDim == 3:
        dispArray   = dataArray[:,4:7]
    else:
        print('Error')
    
    '''
    Mapping Section can be varied for different types of model generation using Abaqus
    '''
    
    minValX  = min(coordArray[:,0])
    maxValX  = max(coordArray[:,0])
    avgValX  = (minValX + maxValX) / 2.0
    longValX = maxValX - minValX
    if longValX < 30:
        minX     = avgValX - 50
        maxX     = avgValX + 50 
    else:
        minX     = avgValX - (longValX*3)/2.0
        maxX     = avgValX + (longValX*3)/2.0
    
   
    # Define map size
    minY = 50
    maxY = 150
    
    if numDim == 3:
        minZ = min(coordArray[:,2])
        maxZ = max(coordArray[:,2])
    else:
        pass
   
    lenX = int(maxX - minX)
    lenY = int(maxY - minY)
    lenStepX = lenX / numElem[1]
    lenStepY = lenY / numElem[0]
   
    mappedData = zeros((numElem[0], numElem[1], 5))
    
    '''
    -----> X
    |
    |
    V Y
    numDim[0] = number of elements for Y direction (Row)
    numDim[1] = number of elements for X direction (Column)
    '''     
    for j in range(numElem[1]):
        # X variation
        xLim = minX + lenStepX*(j)
        for i in range(numElem[0]):
            # Y variation
            yLim = maxY - lenStepY*(i)
            # Initialize 'tempData' at each local mapping area
            tempDataList_1 = []  # Ux
            tempDataList_2 = []  # Uy
            tempDataList_3 = []  # ex
            tempDataList_4 = []  # ey
            tempDataList_5 = []  # exy
            #tempDataList_3 = []
            for k in range(numData):
                xCoord = coordArray[k,0]
                yCoord = coordArray[k,1]
                if xCoord > xLim and xCoord <= xLim + lenStepX:
                    if yCoord <= yLim and yCoord > yLim - lenStepY:
                        tempDataList_1.append(dispArray[k,0])
                        tempDataList_2.append(dispArray[k,1])
                        tempDataList_3.append(strainArray[k,0])
                        tempDataList_4.append(strainArray[k,1])
                        tempDataList_5.append(strainArray[k,3])
                    else:
                        pass
                else:
                    pass
            if len(tempDataList_1) != 0:
                refVal = average(tempDataList_1)
                mappedData[i,j,0] = refVal
            
            if len(tempDataList_2) != 0:
                refVal = average(tempDataList_2)
                mappedData[i,j,1] = refVal
            
            if len(tempDataList_3) != 0:
                refVal = average(tempDataList_3)
                mappedData[i,j,2] = refVal    
            
            if len(tempDataList_4) != 0:
                refVal = average(tempDataList_4)
                mappedData[i,j,3] = refVal    
            
            if len(tempDataList_5) != 0:
                refVal = average(tempDataList_5)
                mappedData[i,j,4] = refVal        
            #if len(tempDataList_3) != 0:
            #    refVal = average(tempDataList_3)
            #    mappedData[i,j,2] = refVal  
    return mappedData, longValX

def elemMapping(dataArray, numDim, numElem):
    ''' 
    Data import from FEM result
    '''
    numData     = len(dataArray[:,0])
    targetID    = dataArray[:,0]
    coordArray  = dataArray[:, 1:3]
    #strainArray = dataArray[:, 5:9]
    if numDim == 2:
        dispArray   = dataArray[:,3]
    elif numDim == 3:
        dispArray   = dataArray[:,4:7]
    else:
        print('Error')
    
    '''
    Mapping part
    '''
    # Define map size
    minValX  = min(coordArray[:,0])
    maxValX  = max(coordArray[:,0])
    avgValX  = (minValX + maxValX) / 2.0
    longValX = maxValX - minValX
    
    if longValX < 30:
        minX     = avgValX - 50
        maxX     = avgValX + 50 
    else:
        minX     = avgValX - (longValX*3)/2.0
        maxX     = avgValX + (longValX*3)/2.0
    
    #if longValX < 100:
    #    minX = avgValX - 50
    #    maxX = avgValX + 50
    #else:
    #    minX = avgValX - 100
    #    maxX = avgValX + 100
    
    minY = 50
    maxY = 150
    
    if numDim == 3:
        minZ = min(coordArray[:,2])
        maxZ = max(coordArray[:,2])
    else:
        pass
   
    lenX = int(maxX - minX)
    lenY = int(maxY - minY)
    lenStepX = lenX / numElem[1]
    lenStepY = lenY / numElem[0]
   
    mappedData = zeros((numElem[0], numElem[1], 1))
    
    '''
    -----> X
    |
    |
    V Y
    numDim[0] = number of elements for Y direction (Row)
    numDim[1] = number of elements for X direction (Column)
    '''    
    for j in range(numElem[1]):
        # X variation
        xLim = minX + lenStepX*(j)
        for i in range(numElem[0]):
            # Y variation
            yLim = maxY - lenStepY*(i)
            # Initialize 'tempData' at each local mapping area
            tempDataList_1 = []  # Ux

            for k in range(numData):
                xCoord = coordArray[k,0]
                yCoord = coordArray[k,1]
                if xCoord > xLim and xCoord <= xLim + lenStepX:
                    if yCoord <= yLim and yCoord > yLim - lenStepY:
                        tempDataList_1.append(dispArray[k])
                    else:
                        pass
                else:
                    pass
            if len(tempDataList_1) != 0:
                refVal = average(tempDataList_1)
                mappedData[i,j,0] = refVal

    return mappedData, longValX

In [ ]:
# MEDIAN FILTER FUNCTIONS
'''
-----> X
|
|
V Y
'''
def latFilter(data):
    nely, nelx = shape(data)
    count = 0
    for i in range(nely):
        for j in range(nelx):
            if data[i, j] == 0:
                if j !=0 and j != nelx-1:
                    if data[i,j-1]!=0 and data[i, j+1] !=0:
                        newVal = (data[i,j-1]+data[i,j+1])/2.0
                        data[i,j] = newVal
                        count = count + 1
                    else:
                        pass
                else:
                    pass
            else:
                pass
    
    return data

def vertFilter(data):
    nely, nelx = shape(data)
    for i in range(nely):
        for j in range(nelx):
            if data[i, j] == 0:
                if i !=0 and data[i-1,j]!=0:
                    if i != nely-1:
                        if data[i+1, j] !=0:
                            newVal = (data[i-1,j]+data[i+1,j])/2.0
                            data[i,j] = newVal
                        else:
                            pass
                    else:
                        pass
                else:
                    pass

    return data

In [ ]:
# ACTUAL RUNNING TEST

nodeDataNameList = ['Ux_','Uy_','ex_','ey_','exy_']


# DATA PROPERTIES AND MAPING SIZE

startNum     = 0
numCase      = 25
numDim       = 2
mappingSize  = [80,80]    

xlist        = linspace(0,1,mappingSize[1])
ylist        = linspace(0,1,mappingSize[0])
X, Y         = np.meshgrid(xlist, ylist)

fileLocation = '.\\rawData\\'
fileSaveLoc  = '.\\ppData\\'

count = 0
countList = []
for i in range(numCase):
    deletOption = False
    
    # ELEMENTAL DATA LOADING: MATERIAL DENSITY
    elemCoordName  = fileLocation + 'elemCoord_' + str(i+startNum) + '.dat'
    elemDensName   = fileLocation + 'matDen_' + str(i+startNum) + '.dat'
    
    elemCoordArray = genfromtxt(elemCoordName,delimiter='\t')
    elemDensArray  = genfromtxt(elemDensName, delimiter='\t')
    
    # ELEM DATA ARRAY GEN
    elemDataArray        = zeros((len(elemCoordArray),4))
    elemDataArray[:,0:3] = elemCoordArray
    elemDataArray[:,3]   = elemDensArray[:,1]
    
    # ELEM DATA MAPPING
    mapElemData, longValXELEM = elemMapping(elemDataArray, numDim, mappingSize)
    
    if longValXELEM <= 30:
        mapElemData = mapElemDataSaved
        count += 1
        countList.append(i)
    else:
        pass

    # ELEM DATA FILTERING
    filtElemDataTemp   = vertFilter(latFilter(mapElemData[:,:,0]))
    filtElemData       = flipud(filtElemDataTemp)
    
    # ELEM DATA FIGURE SAVING
    if i % 200 ==0:
        ppDensFigName   = fileSaveLoc + 'matDen_'  + str(mappingSize[0])+ 'x' +str(mappingSize[1]) + '_' + str(i+startNum) +'.png'
        ppDensFigTitle  = 'matDen_' + str(i+startNum) + '_' + str(mappingSize[0])
        fig_elem,ax=plt.subplots(1,1)
        cp = ax.contourf(X, Y, filtElemData)
        fig_elem.set_size_inches(8, 7)
        ax.set_title(ppDensFigTitle)
        fig_elem.colorbar(cp) # Add a colorbar to a plot
        plt.savefig(ppDensFigName, dpi = 100)
        plt.show()
    #else:
    #    pass
    
    # ELEM DATA SAVING
    ppDensFileName   = fileSaveLoc + 'matDen_' + str(mappingSize[0])+ 'x' +str(mappingSize[1]) + '_' + str(i+startNum) + '.pt'
    torchDensTensor  = nn.from_numpy(filtElemData.copy())
    nn.save(torchDensTensor, ppDensFileName)
    
    # NODAL DATA LOADING: DISPLACEMENT & STRAINS
    nodeFileName = fileLocation + 'nodalData_' + str(i+startNum) + '.dat'
    nodeDataArray= genfromtxt(nodeFileName, delimiter='\t')   
    
    for j in range(len(nodeDataNameList)):
        # NODAL DATA MAPING
        mapNodalData, longValXNodal = nodalMapping(nodeDataArray, numDim, mappingSize)
        if longValXELEM <= 30:
            mapNodalData = mapNodalDataSaved
        else:
            pass
        # NODAL DATA FILTERING
        filtNodalDataTemp = vertFilter(latFilter(mapNodalData[:,:,j]))
        filtNodalData     = flipud(filtNodalDataTemp)
        
        #if i% 200 ==0:
        #    nodalFigName     = fileSaveLoc + nodeDataNameList[j]  + str(mappingSize[0])+ 'x' +str(mappingSize[1]) + '_' + str(i+startNum) + '.png'
        #    nodalFigTitle    = nodeDataNameList[j]  +  str(mappingSize[0])+ 'x' +str(mappingSize[1]) +  '_' + str(i+startNum)
        #    fig_nodal,bx = plt.subplots(1,1)
        #    dp = bx.contourf(X, Y, filtNodalData)
        #    bx.set_title(nodalFigTitle)
        #    fig_nodal.colorbar(dp) # Add a colorbar to a plot
        #    fig_nodal.set_size_inches(8, 7)
        #    plt.savefig(nodalFigName, dpi = 100)
        #    plt.show()

        # NODAL DATA SAVING
        ppNodalFileName = fileSaveLoc + nodeDataNameList[j] + str(mappingSize[0])+ 'x' +str(mappingSize[1]) + '_' + str(i+startNum) + '.pt'
        torchNodalTenor = nn.from_numpy(filtNodalData.copy())
        nn.save(torchNodalTenor, ppNodalFileName)
    
    
    if i%50 == 0 and longValXELEM > 30:
        mapElemDataSaved  = mapElemData
        mapNodalDataSaved = mapNodalData
    else:
        pass

    # PROGRESS PRINTING
    if (i+1)%50 ==0:
        now = datetime.datetime.now()
        date_time = now.strftime("%d/%m/%Y %H:%M:%S")
        print(f'Progress {i+1}/{numCase} // Time: {date_time}')
        print(countList)
    else:
        pass